# Deep Learning with Convolutional Neural Network (CNN, convnet)

We continue with the Fashion MNIST dataset. Last week we tried a traditional densely-connected network that resulted in test accuracy of 88.1%. It will be hard, however, to reach 90% accuracy with traditional models.

Today we try deep learning: we'll build a *CNN* (*Convolutional Neural Network*, also referred to as *convnet*). Even though our convnet will be very basic, its accuracy will still easily pass 90% for this dataset.

This code is based on the example in Chapter 5, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by François Chollet. The changes are:
1. I use the Fashion MNIST dataset rather than the (digit) MNIST dataset, so we can compare with code "`NN_using_Keras.ipynb`" that we tried last week.
2. Due to updates to TensorFlow, the code in Chollet's book no longer works. I updated the code to make it work.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
keras.__version__

'2.4.0'

## Building a convnet: the feature extraction layers

The code below show you what a basic convnet looks like. It's a stack of `Conv2D` and `MaxPooling2D` layers. We'll see in a minute what they do concretely. 

Importantly, a convnet takes as input tensors of shape `(image_height, image_width, image_channels)` (not including the batch dimension). 
In our case, we will configure our convnet to process inputs of size `(28, 28, 1)`, which is the format of MNIST images. We do this via 
passing the argument `input_shape=(28, 28, 1)` to our first layer.

In [2]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Let's display the architecture of our convnet so far:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


You can see above that the output of every `Conv2D` and `MaxPooling2D` layer is a 3D tensor of shape `(height, width, channels)`. The width 
and height dimensions tend to shrink as we go deeper in the network. The number of channels is controlled by the first argument passed to 
the `Conv2D` layers (e.g. 32 or 64).



## Building a convnet: the classification layers

The next step would be to feed our last output tensor (of shape `(3, 3, 64)`) into a densely-connected classifier network like those you are 
already familiar with: a stack of `Dense` layers. These classifiers process vectors, which are 1D, whereas our output so far is a 3D tensor. 
So first, we will have to flatten our 3D outputs to 1D, and then add a few `Dense` layers on top:

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation=tf.nn.relu))
model.add(layers.Dense(10, activation=tf.nn.softmax))
# model.add(layers.Dense(10))

We are going to do 10-way classification, so we use a final layer with 10 outputs and a softmax activation. Now here's what our network 
looks like:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

As you can see, our `(3, 3, 64)` outputs were flattened into vectors of shape `(576,)`, before going through two `Dense` layers.



# Training the model

Now, let's train our convnet on the Fashion MNIST dataset. Note that a lot code we used last week can be reused for this part.

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images / 255.0

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images / 255.0

# train_labels = to_categorical(train_labels)
# test_labels = to_categorical(test_labels)

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
model.compile(optimizer='adam', 
#              loss='categorical_crossentropy',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

Just to make sure: check Colab menu "Runtime"/"Change runtime type", and make sure the choice for "Hardware accelerator" is "GPU". With GPU, the next line of code takes about 1 minute; without GPU, easily 10 minutes.

In [ ]:
model.fit(train_images, train_labels, epochs=20)

Epoch 1/20
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1321 - accuracy: 0.9508
Epoch 2/20
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1217 - accuracy: 0.9544
Epoch 3/20
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1127 - accuracy: 0.9578
Epoch 4/20
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1046 - accuracy: 0.9610
Epoch 5/20
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0975 - accuracy: 0.9630
Epoch 6/20
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0914 - accuracy: 0.9658
Epoch 7/20
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0844 - accuracy: 0.9684
Epoch 8/20
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0796 - accuracy: 0.9698
Epoch 9/20
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0757 - accuracy: 0.9719
Epoch 10/20
1875/1875 [==============================] - 10s 5ms/step - l

Let's evaluate the model on the test data:

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.5436 - accuracy: 0.9077


While our densely-connected network from last week had a test accuracy of 88.1%, our basic convnet has a test accuracy of 91.0%. Therefore, we decreased the error rate from 11.9% to 9.0% -- a significant improvement of (11.9-9.0)/11.9 = 24%! 

In [ ]:
(11.9-9.0)/11.9